<a href="https://colab.research.google.com/github/jiw3026/SML/blob/main/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
train=pd.read_csv('/content/drive/MyDrive/dacon/train.csv')
test=pd.read_csv('/content/drive/MyDrive/dacon/test.csv')

In [4]:
train.isnull().sum()

id                          0
hour                        0
hour_bef_temperature        2
hour_bef_precipitation      2
hour_bef_windspeed          9
hour_bef_humidity           2
hour_bef_visibility         2
hour_bef_ozone             76
hour_bef_pm10              90
hour_bef_pm2.5            117
count                       0
dtype: int64

In [5]:
test.isnull().sum()

id                         0
hour                       0
hour_bef_temperature       1
hour_bef_precipitation     1
hour_bef_windspeed         1
hour_bef_humidity          1
hour_bef_visibility        1
hour_bef_ozone            35
hour_bef_pm10             37
hour_bef_pm2.5            36
dtype: int64

In [6]:
train.fillna(0,inplace=True)
test.fillna(0,inplace=True)

In [7]:
colL=['hour','hour_bef_temperature','hour_bef_precipitation','hour_bef_windspeed','hour_bef_humidity','hour_bef_visibility','hour_bef_visibility','hour_bef_ozone','hour_bef_pm10','hour_bef_pm2.5','count']

train_iqr=train.copy()
for col in colL:
  Q1=train_iqr[col].quantile(.25)
  Q3=train_iqr[col].quantile(.75)
  IQR =Q3-Q1
  train_del=train_iqr[((Q1-1.5*IQR)>train_iqr[col])|((Q3+1.5*IQR)<train_iqr[col])]
  train_iqr=train_iqr.drop(train_del.index,axis=0)
train_iqr

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,count
1,6,13,20.1,0.0,1.4,48.0,916.0,0.042,73.0,40.0,159.0
2,7,6,13.9,0.0,0.7,79.0,1382.0,0.033,32.0,19.0,26.0
5,13,2,13.6,0.0,1.7,80.0,1073.0,0.027,34.0,15.0,39.0
6,14,3,10.6,0.0,1.5,58.0,1548.0,0.038,62.0,33.0,23.0
7,16,21,16.0,0.0,6.0,21.0,1961.0,0.050,90.0,28.0,146.0
...,...,...,...,...,...,...,...,...,...,...,...
1454,2174,4,16.8,0.0,1.6,53.0,2000.0,0.031,37.0,27.0,21.0
1455,2175,3,10.8,0.0,3.8,45.0,2000.0,0.039,34.0,19.0,20.0
1456,2176,5,18.3,0.0,1.9,54.0,2000.0,0.009,30.0,21.0,22.0
1457,2178,21,20.7,0.0,3.7,37.0,1395.0,0.082,71.0,36.0,216.0


In [8]:
train_x=train_iqr.drop(['count'],axis=1)
train_y=train_iqr['count']
X_train,X_test,y_train,y_test=train_test_split(train_x,train_y,test_size=0.25,random_state=42)

In [9]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
scaler.fit(X_train)
X_train=scaler.transform(X_train)

In [10]:
X_test=scaler.transform(X_test)

In [11]:
#train데이터정확도
from sklearn.svm import SVR
model=SVR(kernel='poly')
model.fit(X_train,y_train)
pred_train=model.predict(X_train)
model.score(X_train,y_train)

0.4227418049315229

In [12]:
#test데이터정확도
pred_test=model.predict(X_test)
model.score(X_test,y_test)

0.4205565595047416

In [13]:
#RMSE
from sklearn.metrics import mean_squared_error
MSE_train=mean_squared_error(y_train,pred_train)
MSE_test=mean_squared_error(y_test,pred_test)
print('train데이터 :',np.sqrt(MSE_train))
print('test데이터 :',np.sqrt(MSE_test))

train데이터 : 59.137950688476245
test데이터 : 63.729168341972944


In [25]:
#하이퍼파라미터 조정 : Grid Search
from sklearn.model_selection import GridSearchCV
param_grid={'C':[0.01,0.1,1,10],'kernel':['rbf'],'gamma':[0.01,0.1,1,10]}
grid_search=GridSearchCV(SVR(),param_grid,cv=5)
grid_search.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [0.01, 0.1, 1, 10], 'gamma': [0.01, 0.1, 1, 10],
                         'kernel': ['rbf']})

In [26]:
#최적의 하이퍼파라미터
print('Best Parameter : {}'.format(grid_search.best_params_))
#train데이터 정확도
print('Best Train Score : {:.4f}'.format(grid_search.best_score_))
#test데이터 정확도
print('Best Test Score : {:.4f}'.format(grid_search.score(X_test,y_test)))

Best Parameter : {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
Best Train Score : 0.5602
Best Test Score : 0.6539


In [27]:
scores_df=pd.DataFrame(grid_search.cv_results_)
scores_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.095511,0.053319,0.021786,0.014743,0.01,0.01,rbf,"{'C': 0.01, 'gamma': 0.01, 'kernel': 'rbf'}",-0.007349,-0.097539,0.001769,0.003404,-0.000959,-0.020135,0.038875,10
1,0.040841,0.004037,0.010075,0.000149,0.01,0.1,rbf,"{'C': 0.01, 'gamma': 0.1, 'kernel': 'rbf'}",-0.002357,-0.090943,0.006699,0.008254,0.004290,-0.014811,0.038238,9
2,0.037863,0.001230,0.011062,0.001753,0.01,1,rbf,"{'C': 0.01, 'gamma': 1, 'kernel': 'rbf'}",-0.011531,-0.101849,-0.001961,-0.000631,-0.005076,-0.024209,0.039002,13
3,0.048724,0.000781,0.009986,0.000107,0.01,10,rbf,"{'C': 0.01, 'gamma': 10, 'kernel': 'rbf'}",-0.011939,-0.102311,-0.002318,-0.000989,-0.005486,-0.024609,0.039035,16
4,0.038475,0.001165,0.009996,0.000151,0.1,0.01,rbf,"{'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'}",0.029249,-0.052990,0.037369,0.039989,0.035669,0.017857,0.035600,7
5,0.037604,0.001481,0.011111,0.002160,0.1,0.1,rbf,"{'C': 0.1, 'gamma': 0.1, 'kernel': 'rbf'}",0.072520,0.001276,0.077840,0.081257,0.077258,0.062030,0.030505,6
6,0.036867,0.000756,0.010208,0.000222,0.1,1,rbf,"{'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}",-0.008065,-0.097432,0.001282,0.002603,-0.001263,-0.020575,0.038604,11
7,0.049777,0.001863,0.010465,0.000781,0.1,10,rbf,"{'C': 0.1, 'gamma': 10, 'kernel': 'rbf'}",-0.012020,-0.102045,-0.002286,-0.000971,-0.005359,-0.024536,0.038942,15
8,0.037329,0.000451,0.010024,0.000278,1,0.01,rbf,"{'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}",0.267963,0.206568,0.267989,0.277432,0.267551,0.257500,0.025737,4
9,0.037859,0.002367,0.011158,0.002315,1,0.1,rbf,"{'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}",0.367288,0.350104,0.375366,0.383156,0.370506,0.369284,0.010980,3


In [39]:
scores_df['rmse']=np.sqrt(abs(scores_df['mean_test_score']))

In [51]:
rmse_df=scores_df.loc[:,['param_C','param_gamma','param_kernel','mean_test_score','rmse']]
rmse_df

,param_C,param_gamma,param_kernel,mean_test_score,rmse
0,0.01,0.01,rbf,-0.020135,0.141897
1,0.01,0.1,rbf,-0.014811,0.121701
2,0.01,1,rbf,-0.024209,0.155593
3,0.01,10,rbf,-0.024609,0.156871
4,0.1,0.01,rbf,0.017857,0.133631
5,0.1,0.1,rbf,0.062030,0.249058
6,0.1,1,rbf,-0.020575,0.143441
7,0.1,10,rbf,-0.024536,0.156640
8,1,0.01,rbf,0.257500,0.507445
9,1,0.1,rbf,0.369284,0.607687
